Capstone Project - Recipe/Meal Recommendation System

In [1]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
import re
import numpy as np
import pickle
from nltk.tokenize import RegexpTokenizer, word_tokenize
from nltk.stem import WordNetLemmatizer

In [2]:
user_data = pd.read_csv('data/RAW_interactions.csv')
user_data.head()

,user_id,recipe_id,date,rating,review
0,38094,40893,2003-02-17,4,Great with a salad. Cooked on top of stove for...
1,1293707,40893,2011-12-21,5,"So simple, so delicious! Great for chilly fall..."
2,8937,44394,2002-12-01,4,This worked very well and is EASY. I used not...
3,126440,85009,2010-02-27,5,I made the Mexican topping and took it to bunk...
4,57222,85009,2011-10-01,5,"Made the cheddar bacon topping, adding a sprin..."


In [3]:
recipe_data = pd.read_csv('data/RAW_recipes.csv')
recipe_data.head()

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8


In [4]:
recipe_df = recipe_data.drop(columns=['contributor_id', 'submitted', 'tags', 'nutrition', 'description', 'ingredients', 'n_steps', 'steps', 'n_ingredients'])
recipe_df.head()

,name,id,minutes
0,arriba baked winter squash mexican style,137739,55
1,a bit different breakfast pizza,31490,30
2,all in the kitchen chili,112140,130
3,alouette potatoes,59389,45
4,amish tomato ketchup for canning,44061,190


In [5]:
from surprise import Reader, Dataset
# read in values as Surprise dataset 

reader = Reader()
recipe_df = Dataset.load_from_df(recipe_df, reader)


In [6]:
print(recipe_df)

In [7]:
recipes_PP = pd.read_csv('data/PP_recipes.csv')
recipes_PP.head()

,id,i,name_tokens,ingredient_tokens,steps_tokens,techniques,calorie_level,ingredient_ids
0,424415,23,"[40480, 37229, 2911, 1019, 249, 6878, 6878, 28...","[[2911, 1019, 249, 6878], [1353], [6953], [153...","[40480, 40482, 21662, 481, 6878, 500, 246, 161...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",0,"[389, 7655, 6270, 1527, 3406]"
1,146223,96900,"[40480, 18376, 7056, 246, 1531, 2032, 40481]","[[17918], [25916], [2507, 6444], [8467, 1179],...","[40480, 40482, 729, 2525, 10906, 485, 43, 8393...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",0,"[2683, 4969, 800, 5298, 840, 2499, 6632, 7022,..."
2,312329,120056,"[40480, 21044, 16954, 8294, 556, 10837, 40481]","[[5867, 24176], [1353], [6953], [1301, 11332],...","[40480, 40482, 8240, 481, 24176, 296, 1353, 66...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...",1,"[1257, 7655, 6270, 590, 5024, 1119, 4883, 6696..."
3,74301,168258,"[40480, 10025, 31156, 40481]","[[1270, 1645, 28447], [21601], [27952, 29471, ...","[40480, 40482, 5539, 21601, 1073, 903, 2324, 4...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,"[7940, 3609, 7060, 6265, 1170, 6654, 5003, 3561]"
4,76272,109030,"[40480, 17841, 252, 782, 2373, 1641, 2373, 252...","[[1430, 11434], [1430, 17027], [1615, 23, 695,...","[40480, 40482, 14046, 1430, 11434, 488, 17027,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",0,"[3484, 6324, 7594, 243]"


In [8]:
users_PP = pd.read_csv('data/PP_users.csv')
users_PP.head()

,u,techniques,items,n_items,ratings,n_ratings
0,0,"[8, 0, 0, 5, 6, 0, 0, 1, 0, 9, 1, 0, 0, 0, 1, ...","[1118, 27680, 32541, 137353, 16428, 28815, 658...",31,"[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 4.0, 4.0, ...",31
1,1,"[11, 0, 0, 2, 12, 0, 0, 0, 0, 14, 5, 0, 0, 0, ...","[122140, 77036, 156817, 76957, 68818, 155600, ...",39,"[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, ...",39
2,2,"[13, 0, 0, 7, 5, 0, 1, 2, 1, 11, 0, 1, 0, 0, 1...","[168054, 87218, 35731, 1, 20475, 9039, 124834,...",27,"[3.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 5.0, ...",27
3,3,"[498, 13, 4, 218, 376, 3, 2, 33, 16, 591, 10, ...","[163193, 156352, 102888, 19914, 169438, 55772,...",1513,"[5.0, 5.0, 5.0, 5.0, 4.0, 4.0, 5.0, 5.0, 5.0, ...",1513
4,4,"[161, 1, 1, 86, 93, 0, 0, 11, 2, 141, 0, 16, 0...","[72857, 38652, 160427, 55772, 119999, 141777, ...",376,"[5.0, 5.0, 5.0, 5.0, 4.0, 4.0, 5.0, 4.0, 5.0, ...",376


In [9]:
users_PP.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25076 entries, 0 to 25075
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   u           25076 non-null  int64 
 1   techniques  25076 non-null  object
 2   items       25076 non-null  object
 3   n_items     25076 non-null  int64 
 4   ratings     25076 non-null  object
 5   n_ratings   25076 non-null  int64 
dtypes: int64(3), object(3)
memory usage: 1.1+ MB


In [10]:
#users_PP['ratings'] = users_PP['ratings'].astype(float)
user_df = user_data.drop(columns=['date', 'review'])
#users_PP = users_PP.rename(columns = {'u': 'user', 'items':'recipes'})
user_df.head()

,user_id,recipe_id,rating
0,38094,40893,4
1,1293707,40893,5
2,8937,44394,4
3,126440,85009,5
4,57222,85009,5


In [11]:
user_df = Dataset.load_from_df(user_df, reader)
user_df

In [12]:
pickled_map = pd.read_pickle('data/ingr_map.pkl')
pickled_map

,raw_ingr,raw_words,processed,len_proc,replaced,count,id
0,"medium heads bibb or red leaf lettuce, washed,...",13,"medium heads bibb or red leaf lettuce, washed,...",73,lettuce,4507,4308
1,mixed baby lettuces and spring greens,6,mixed baby lettuces and spring green,36,lettuce,4507,4308
2,romaine lettuce leaf,3,romaine lettuce leaf,20,lettuce,4507,4308
3,iceberg lettuce leaf,3,iceberg lettuce leaf,20,lettuce,4507,4308
4,red romaine lettuce,3,red romaine lettuce,19,lettuce,4507,4308
...,...,...,...,...,...,...,...
11654,soybeans,1,soybean,7,soybean,31,6702
11655,goose,1,goose,5,goose,8,3318
11656,ajwain,1,ajwain,6,ajwain,13,47
11657,brinjals,1,brinjal,7,brinjal,2,750


## Data Exploration

In [14]:
user_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1132367 entries, 0 to 1132366
Data columns (total 5 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   user_id    1132367 non-null  int64 
 1   recipe_id  1132367 non-null  int64 
 2   date       1132367 non-null  object
 3   rating     1132367 non-null  int64 
 4   review     1132198 non-null  object
dtypes: int64(3), object(2)
memory usage: 43.2+ MB


In [15]:
print(len(user_data['user_id'].unique()))

226570


There are 226,570 unique users in this dataset

In [16]:
recipe_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231637 entries, 0 to 231636
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   name            231636 non-null  object
 1   id              231637 non-null  int64 
 2   minutes         231637 non-null  int64 
 3   contributor_id  231637 non-null  int64 
 4   submitted       231637 non-null  object
 5   tags            231637 non-null  object
 6   nutrition       231637 non-null  object
 7   n_steps         231637 non-null  int64 
 8   steps           231637 non-null  object
 9   description     226658 non-null  object
 10  ingredients     231637 non-null  object
 11  n_ingredients   231637 non-null  int64 
dtypes: int64(5), object(7)
memory usage: 21.2+ MB


In [17]:
recipe_data = recipe_data.drop(columns=['contributor_id', 'submitted', 'nutrition', 'steps'])

In [18]:
print(len(recipe_data['id'].unique()))

231637


There are 231,637 unique recipes in this dataset.

With 226,570 users and 231,637 recipes, there are less users than there are recipes. Therefore, it is probably best for our recommender system to be user-user based.

## Preprocessing

The text data in description needs to be cleaned to ensure all punctuation is removed and words are all lowercase. The text data in the other columns looks pretty well cleaned already, but we still need to remove stopwords.

In [19]:
# Creating a function to perform cleaning steps at once
stopwords_list = stopwords.words('english')

no_bad_chars = re.compile('[!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n - ]')
no_nums = re.compile('[\d-]')

def clean_text(text):
    #text = no_nums.sub('', text) 
    text = no_bad_chars.sub(' ', text) 
    text = text.lower() 
    text = ' '.join(word for word in text.split() if word not in stopwords_list)
    return text


In [19]:
recipe_data['description'] = recipe_data['description'].astype(str)
descr_cleaned = recipe_data['description'].apply(clean_text)
descr_cleaned

0         autumn favorite time year cook recipe prepared...
1         recipe calls crust prebaked bit adding ingredi...
2         modified version 'mom's' chili hit 2004 christ...
3         super easy great tasting make ahead side dish ...
4         dh's amish mother raised recipe much prefers s...
                                ...                        
231632    delicious soup originally found better homes g...
231633                      spice mix make taste buds dance
231634                             deviled eggs cajun style
231635    i've heard 'cookies design' company never trie...
231636    i've heard 'cookies design' company never trie...
Name: description, Length: 231637, dtype: object

In [20]:
recipe_data['name'] = recipe_data['name'].astype(str)
names_cleaned = recipe_data['name'].apply(clean_text)
names_cleaned

0         arriba baked winter squash mexican style
1                    bit different breakfast pizza
2                                    kitchen chili
3                                alouette potatoes
4                     amish tomato ketchup canning
                            ...                   
231632                                 zydeco soup
231633                            zydeco spice mix
231634                   zydeco ya ya deviled eggs
231635                cookies design cookies stick
231636     cookies design sugar shortbread cookies
Name: name, Length: 231637, dtype: object

In [21]:
recipe_data['tags'] = recipe_data['tags'].astype(str)
tags_cleaned = recipe_data['tags'].apply(clean_text)
tags_cleaned

0         '60 minutes less' 'time make' 'course' 'main i...
1         '30 minutes less' 'time make' 'course' 'main i...
2         'time make' 'course' 'preparation' 'main dish'...
3         '60 minutes less' 'time make' 'course' 'main i...
4         'weeknight' 'time make' 'course' 'main ingredi...
                                ...                        
231632    'ham' '60 minutes less' 'time make' 'course' '...
231633    '15 minutes less' 'time make' 'course' 'prepar...
231634    '60 minutes less' 'time make' 'course' 'main i...
231635    '30 minutes less' 'time make' 'course' 'prepar...
231636    '30 minutes less' 'time make' 'course' 'prepar...
Name: tags, Length: 231637, dtype: object

In [22]:
recipe_data['tags'] = recipe_data['tags'].astype(str)
tags_cleaned = recipe_data['tags'].apply(clean_text)
tags_cleaned

0         '60 minutes less' 'time make' 'course' 'main i...
1         '30 minutes less' 'time make' 'course' 'main i...
2         'time make' 'course' 'preparation' 'main dish'...
3         '60 minutes less' 'time make' 'course' 'main i...
4         'weeknight' 'time make' 'course' 'main ingredi...
                                ...                        
231632    'ham' '60 minutes less' 'time make' 'course' '...
231633    '15 minutes less' 'time make' 'course' 'prepar...
231634    '60 minutes less' 'time make' 'course' 'main i...
231635    '30 minutes less' 'time make' 'course' 'prepar...
231636    '30 minutes less' 'time make' 'course' 'prepar...
Name: tags, Length: 231637, dtype: object

In [23]:
basic_token_pattern = r"(?u)\b\w\w+\b"
tokenizer = RegexpTokenizer(basic_token_pattern)

I imagine that certain recipes that fall under different diet categories, such as vegan/vegetarian, gluten-free, etc., would have that listed as a tag...
- similarity metric might do this on its own?

In [24]:
# tokenize tags:
tags_tokenized = tags_cleaned.apply(tokenizer.tokenize)
tags_tokenized

0         [60, minutes, less, time, make, course, main, ...
1         [30, minutes, less, time, make, course, main, ...
2         [time, make, course, preparation, main, dish, ...
3         [60, minutes, less, time, make, course, main, ...
4         [weeknight, time, make, course, main, ingredie...
                                ...                        
231632    [ham, 60, minutes, less, time, make, course, m...
231633    [15, minutes, less, time, make, course, prepar...
231634    [60, minutes, less, time, make, course, main, ...
231635    [30, minutes, less, time, make, course, prepar...
231636    [30, minutes, less, time, make, course, prepar...
Name: tags, Length: 231637, dtype: object

In [25]:
# tokenize recipe names:
name_tokenized = names_cleaned.apply(tokenizer.tokenize)
name_tokenized.head(10)

0    [arriba, baked, winter, squash, mexican, style]
1                 [bit, different, breakfast, pizza]
2                                   [kitchen, chili]
3                               [alouette, potatoes]
4                  [amish, tomato, ketchup, canning]
5                          [apple, day, milk, shake]
6                           [aww, marinated, olives]
7                 [backyard, style, barbecued, ribs]
8                         [bananas, ice, cream, pie]
9                              [beat, banana, bread]
Name: name, dtype: object

In [26]:
# tokenize recipe descriptions:
descr_tokenized = descr_cleaned.apply(tokenizer.tokenize)
descr_tokenized.head(10)

0    [autumn, favorite, time, year, cook, recipe, p...
1    [recipe, calls, crust, prebaked, bit, adding, ...
2    [modified, version, mom, chili, hit, 2004, chr...
3    [super, easy, great, tasting, make, ahead, sid...
4    [dh, amish, mother, raised, recipe, much, pref...
5                                                [nan]
6    [italian, mil, thoroughly, impressed, non, ita...
7    [recipe, posted, request, originaly, chef, sam...
8                                                [nan]
9                                       [ann, hodgman]
Name: description, dtype: object

Aside from tags, to categorize recipes into different diets I will likely need to manually filter through the ingredients data.

In [21]:
# tokenize ingredients:
recipe_data['ingr_tokenized'] = recipe_data['ingredients'].apply(tokenizer.tokenize)
recipe_data.head()

,name,id,minutes,tags,n_steps,description,ingredients,n_ingredients,ingr_tokenized
0,arriba baked winter squash mexican style,137739,55,"['60-minutes-or-less', 'time-to-make', 'course...",11,autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7,"[winter, squash, mexican, seasoning, mixed, sp..."
1,a bit different breakfast pizza,31490,30,"['30-minutes-or-less', 'time-to-make', 'course...",9,this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6,"[prepared, pizza, crust, sausage, patty, eggs,..."
2,all in the kitchen chili,112140,130,"['time-to-make', 'course', 'preparation', 'mai...",6,this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13,"[ground, beef, yellow, onions, diced, tomatoes..."
3,alouette potatoes,59389,45,"['60-minutes-or-less', 'time-to-make', 'course...",11,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11,"[spreadable, cheese, with, garlic, and, herbs,..."
4,amish tomato ketchup for canning,44061,190,"['weeknight', 'time-to-make', 'course', 'main-...",5,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8,"[tomato, juice, apple, cider, vinegar, sugar, ..."


In [22]:
# how many recipe names include the word 'gluten-free'?
print(len(names_cleaned[names_cleaned.str.contains('gluten free')]))
names_cleaned[names_cleaned.str.contains('gluten free')]

942


586                 miracle honey oatmeal bread gluten free
755                             1 pound gluten free lasagna
1303      3 variations gluten free bread recipe bread ma...
1833           gluten free springfield style cashew chicken
2336      absolutely best gluten free peanut butter cookies
                                ...                        
229936             yummy foolproof gluten free french bread
229958                              yummy gluten free pasta
229959                    yummy gluten free pumpkin muffins
231030                           zucchini bread gluten free
231039                           zucchini bread gluten free
Name: name, Length: 942, dtype: object

This tells us that some ingredients may be vegan/vegetarian but will not read as such (for example 'vegetarian meat').

In [23]:
#most common tags?
#top_tags = tags_tokenized.value_counts()[:10].sort_values(ascending=False)
#top_tags

In [24]:
#create 'diet' column for recipes to categorize them as different diet types
#recipe_data['check'] = None

#recipe_data["diet"] = np.nan

In [25]:
#Lemmatizing


#lemmatizer = WordNetLemmatizer() 
#tokenizer=word_tokenize
#def lem_and_tokenize(document):
#    tokens = tokenizer(document)
#    return [lemmatizer.lemmatize(token) for token in tokens]


#lemm_stopwords = [lemmatizer.lemmatize(word) for word in stopwords_list]
#

In [26]:
#recipe_data['lemmed_ingr'] = recipe_data['ingredients'].apply(lem_and_tokenize)

In [27]:
#recipe_data.head()

Ingredient lists for diet filtering:

In [28]:
meat = ['ham', 'beef', 'meat', 'chicken', 'pork', 'bacon', 'sausage', 'lamb', 'veal', 'turkey', 'steak', 'rib', 'frankfurter', 'duck', 'poultry', 'goat', 'liver', 'hen', 'quail', 'brisket', 'goose']
seafood = ['fish', 'shrimp', 'seafood', 'crab', 'lobster', 'clam', 'oyster', 'scallop', 'mussel', 'cod', 'salmon', 'halibut', 'shellfish', 'roe', 'tuna', 'caviar', 'pollock', 'yellowtail', 'squid', 'calamari', 'octopus', 'crawfish', 'crayfish', 'sardine', 'trout', 'flounder', 'anchovy', 'bass', 'haddock', 'sole' ]
animal_other = ['egg', 'honey']
dairy = ['milk', 'cheese', 'yogurt', 'mayonnaise', 'butter', 'margarine', 'cream']
gluten_free = ['gluten free', 'gluten-free']


vegan = ['ham', 'beef', 'meat', 'chicken', 'pork', 'bacon', 'sausage', 'lamb', 'veal', 'turkey', 'steak', 'rib', 'frankfurter', 'duck', 'poultry', 'goat', 'liver', 'hen', 'quail', 'brisket', 'goose','fish', 'shrimp', 'seafood', 'crab', 'lobster', 'clam', 'oyster', 'scallop', 'mussel', 'cod', 'salmon', 'halibut', 'shellfish', 'roe', 'tuna', 'caviar', 'pollock', 'yellowtail', 'squid', 'calamari', 'octopus', 'crawfish', 'crayfish', 'sardine', 'trout', 'flounder', 'anchovy', 'bass', 'haddock', 'sole','egg', 'honey','milk', 'cheese', 'yogurt', 'mayonnaise', 'butter', 'margarine', 'cream']

vegetarian = ['ham', 'beef', 'meat', 'chicken', 'pork', 'bacon', 'sausage', 'lamb', 'veal', 'turkey', 'steak', 'rib', 'frankfurter', 'duck', 'poultry', 'goat', 'liver', 'hen', 'quail', 'brisket', 'goose','fish', 'shrimp', 'seafood', 'crab', 'lobster', 'clam', 'oyster', 'scallop', 'mussel', 'cod', 'salmon', 'halibut', 'shellfish', 'roe', 'tuna', 'caviar', 'pollock', 'yellowtail', 'squid', 'calamari', 'octopus', 'crawfish', 'crayfish', 'sardine', 'trout', 'flounder', 'anchovy', 'bass', 'haddock', 'sole']


In [75]:
def gf_recipe(recipe_data):
    if any(tags_tokenized.str.contains("gluten-free")):
        return True
    else:
        return False

In [18]:
recipe_data['gf_diet'] = recipe_data.apply(gf_recipe, axis=1)
recipe_data.head()

In [43]:
#recipe_data['vegetarian'] = None
def veg_recipe(recipe_data):
    if recipe_data['tags'].str.contains("vegetarian"):
        recipe_data['vegetarian'] == False
    else:
        recipe_data['vegetarian'] == True

In [ ]:
recipe_data['veg_diet'] = recipe_data.apply(veg_recipe, axis=1)
recipe_data.head()

In [ ]:
#recipe_data['vegan_diet'] = None
def vegan_recipe(recipe_data):
    if recipe_data['tags'].str.contains("vegan"):
        recipe_data['vegetarian'] == False
    else:
        recipe_data['vegetarian'] == True

In [ ]:
recipe_data['vegan_diet'] = recipe_data.apply(vegan_recipe, axis=1)
recipe_data.head()

___

In [119]:
vegan_ = recipe_data[recipe_data['ingredients'].isin(['ham', 'beef', 'meat', 'chicken', 'pork', 'bacon', 'sausage', 'lamb', 'veal', 'turkey', 'steak', 'rib', 'frankfurter', 'duck', 'poultry', 'goat', 'liver', 'hen', 'quail', 'brisket', 'goose','fish', 'shrimp', 'seafood', 'crab', 'lobster', 'clam', 'oyster', 'scallop', 'mussel', 'cod', 'salmon', 'halibut', 'shellfish', 'roe', 'tuna', 'caviar', 'pollock', 'yellowtail', 'squid', 'calamari', 'octopus', 'crawfish', 'crayfish', 'sardine', 'trout', 'flounder', 'anchovy', 'bass', 'haddock', 'sole','egg', 'honey','milk', 'cheese', 'yogurt', 'mayonnaise', 'butter', 'margarine', 'cream'])==False]
#vegetarian_ = ingr_tokenized.isin(['ham', 'beef', 'meat', 'chicken', 'pork', 'bacon', 'sausage', 'lamb', 'veal', 'turkey', 'steak', 'rib', 'frankfurter', 'duck', 'poultry', 'goat', 'liver', 'hen', 'quail', 'brisket', 'goose','fish', 'shrimp', 'seafood', 'crab', 'lobster', 'clam', 'oyster', 'scallop', 'mussel', 'cod', 'salmon', 'halibut', 'shellfish', 'roe', 'tuna', 'caviar', 'pollock', 'yellowtail', 'squid', 'calamari', 'octopus', 'crawfish', 'crayfish', 'sardine', 'trout', 'flounder', 'anchovy', 'bass', 'haddock', 'sole'])==False


## Modeling

In [13]:
from surprise.model_selection import train_test_split

trainset, testset = train_test_split(user_df, test_size=0.2)

In [14]:
#need to determine best model
from surprise.model_selection import cross_validate
from surprise.prediction_algorithms import SVD
from surprise.prediction_algorithms import KNNWithMeans, KNNBasic, KNNBaseline
from surprise.model_selection import GridSearchCV

In [15]:
## Perform a gridsearch with SVD
# ⏰ This cell may take several minutes to run
params = {'n_factors': [20,50,100],
         'reg_all': [0.02, 0.05, 0.1]}
g_s_svd = GridSearchCV(SVD, param_grid=params, n_jobs=-1)
g_s_svd.fit(user_df)


KeyboardInterrupt: 

In [16]:
# print out optimal parameters for SVD after GridSearch
print(g_s_svd.best_score)
print(g_s_svd.best_params)

AttributeError: 'GridSearchCV' object has no attribute 'best_score'

In [ ]:
# cross validating with KNNBasic
knn_basic = KNNBasic(sim_options= {'name': 'pearson', 'user_based':True})
cv_knn_basic = cross_validate(knn_basic, user_df, n_jobs=-1)

In [ ]:
# print out the average RMSE score for the test set
for i in cv_knn_basic.items():
    print(i)
print('-------')
print(np.mean(cv_knn_basic['test_rmse']))

In [ ]:
# cross validating with KNNBaseline
knn_baseline = KNNBaseline(sim_options={'name': 'pearson', 'user_based':True})
cv_knn_baseline = cross_validate(knn_baseline, user_df)

In [ ]:
# print out the average score for the test set
for i in cv_knn_baseline.items():
    print(i)
np.mean(cv_knn_baseline['test_rmse'])

In [17]:
trainset, testset = train_test_split(user_df, test_size=0.2)

In [19]:
# train a model using the new combined DataFrame
#svd_ = SVD(n_factors=20, reg_all=0.1)
#svd_.fit(testset)
from surprise import accuracy

svd = SVD(n_factors=20, n_epochs=10, lr_all=0.005, reg_all=0.1)
svd.fit(trainset)
predictions = svd.test(testset)
print(accuracy.mae(predictions))

MAE:  0.7463
0.7462732956209889


In [31]:
from surprise.prediction_algorithms import knns
from surprise.similarities import cosine, msd, pearson
from surprise import accuracy
svd = SVD(sim_options=pearson)
svd.fit(trainset)


TypeError: __init__() got an unexpected keyword argument 'sim_options'

____

In [ ]:
user_ratings = user_data.pivot(index='user_id', columns='recipe_id', values='rating')
user_ratings.head()

In [ ]:
for col in user_ratings:
    mean = user_ratings[col].mean()
    user_ratings[col] = user_ratings[col].fillna(value=mean)
user_ratings.head()

In [ ]:
import numpy as np
import datetime

In [ ]:
# ⏰ Expect this cell to take several minutes to run
start = datetime.datetime.now()
user_matrix = []
for i, row in enumerate(user_ratings.index):
    u1 = user_ratings[row]
    # Matrix is symetric, so fill in values for previously examined users
    user_distances = [entry[i] for entry in user_matrix] 
    for j, row2 in enumerate(user_ratings.index[i:]):
        u2 = user_ratings[row2]
        d = distance(u1,u2)
        user_distances.append(d)
    user_matrix.append(user_distances)
user_similarities = pd.DataFrame(user_matrix)

end = datetime.datetime.now()
elapsed = end - start
print(elapsed)

user_similarities.head()

In [ ]:
#function from rec. systems lesson, removes previously rated items and orders items for recommendations based on rating (high to low)
#based on avg of closest users ratings
def recommend_recipes(user, user_similarities, user_ratings, df, n_users=20, n_items=10):
    """n is the number of similar users who you wish to use to generate recommendations."""
    # User_Similarities Offset By 1 and Must Remove Current User
    top_n_similar_users = user_similarities[user-1].drop(user-1).sort_values().index[:n_users] 
    # Again, fixing the offset of user_ids
    top_n_similar_users = [i+1 for i in top_n_similar_users] 
    already_rated = set(df[df.user_id == 0].item_id.unique())
    unrated = set(df.item_id.unique()) - already_rated
    projected_user_ratings = user_ratings[user_ratings.index.isin(top_n_similar_users)].mean()[list(unrated)].sort_values(ascending=False)
    return projected_user_ratings[:n_items]

In [ ]:
recommend_movies(1, user_similarities, user_ratings, df)

In [ ]:
# return the top n recommendations using the 
def recommended_recipes(user_ratings,movie_title_df,n):
        for idx, rec in enumerate(user_ratings):
            title = movie_title_df.loc[movie_title_df['movieId'] == int(rec[0])]['title']
            print('Recommendation #', idx+1, ': ', title, '/n')
            n-=1 
            if n==0:
                break
                
recommended_movies(ranked_movies,df_movies,5)

- stemm or lemm text data
- 

limitation: person can be multiple 'diet types' at once